In [18]:
import boto3;
import json;


mediaconvert = boto3.client(
        "mediaconvert",
        endpoint_url="https://" +
        "fkuulejsc.mediaconvert.us-east-2.amazonaws.com")

env_settings = {
    'AccelerationSettings': {
        'Mode': 'DISABLED'
    },
    'BillingTagsSource': 'QUEUE',
    'QueuePrefix': "arn:aws:mediaconvert:us-east-2:693949087897:queues/",
    'Role': "arn:aws:iam::693949087897:role/mediaRole",
    'Endpoint': "https://fkuulejsc.mediaconvert.us-east-2.amazonaws.com"
}

In [19]:
#sincronizzazione resume remoto con notebook
#prendo il resume con i dettagli e il file delle label
!aws s3 sync s3://ahlconsolebucket/tmp ./
!aws s3 sync s3://ahlconsolebucket/utils ./

download: s3://ahlconsolebucket/tmp/modified-resume.json to ./modified-resume.json
                                                              


In [27]:
#trasformo i file json in array di elementi
data_file = open("modified-resume.json")
labels_file = open("labels.json")
data_bin = data_file.read()
labels_bin = labels_file.read()
data = json.loads(data_bin)
labels = json.loads(labels_bin)['labels']
print(labels)

['change', 'esultanza', 'game', 'goal', 'guardalinee', 'palo', 'punizione', 'redcard', 'rigore', 'var', 'yellowcard']


In [42]:
#creo la key del video originale per usarla con media convert
full_key = data[0]['frame_key'].split('/')
frame_key = full_key[len(full_key)-1]
prefix_key = frame_key.split('.')[:-2]
key = 's3://ahlconsolebucket/origin/' + '.'.join(prefix_key) + '.mp4'
print(key)

# name = splitted[-1]
# splitted = name.split('.')

# #Before: video_key = splitted[-2:] . Modified for test (before line 96 was a TypeError)
# video_key = splitted[-3]

# input_file_key = 's3://ahlconsolebucket/origin/' + video_key + '.mp4'

s3://ahlconsolebucket/origin/Football_Red_card_to_Top_Players_2019-1.mp4


In [44]:
#creo un ciclo che controlla tutti i dettagli recuperati dal resume
#per ogni dettaglio controlla se è uman e show
#imposta la cartella di destinazione con il nome della label associata a quel ritaglio
#con i dettagli recuperati effettua il clipping
H264 = dict(
                InterlaceMode="PROGRESSIVE", NumberReferenceFrames=3,
                Syntax="DEFAULT", Softness=0, GopClosedCadence=1,
                GopSize=90, Slices=1, GopBReference="DISABLED",
                SlowPal="DISABLED", SpatialAdaptiveQuantization="ENABLED",
                TemporalAdaptiveQuantization="ENABLED",
                FlickerAdaptiveQuantization="DISABLED",
                EntropyEncoding="CABAC", Bitrate=3195,
                FramerateControl="INITIALIZE_FROM_SOURCE",
                RateControlMode="CBR", CodecProfile="MAIN",
                Telecine="NONE", MinIInterval=0,
                AdaptiveQuantization="HIGH", CodecLevel="AUTO",
                FieldEncoding="PAFF", SceneChangeDetect="ENABLED",
                QualityTuningLevel="SINGLE_PASS",
                FramerateConversionAlgorithm="DUPLICATE_DROP",
                UnregisteredSeiTimecode="DISABLED", GopSizeUnits="FRAMES",
                ParControl="INITIALIZE_FROM_SOURCE",
                NumberBFramesBetweenReferenceFrames=2,
                RepeatPps="DISABLED", DynamicSubGop="STATIC"
            )
env_settings = {
    'AccelerationSettings': {
        'Mode': 'DISABLED'
    },
    'BillingTagsSource': 'QUEUE',
    'QueuePrefix': "arn:aws:mediaconvert:us-east-2:693949087897:queues/",
    'Role': "arn:aws:iam::693949087897:role/mediaRole",
    'Endpoint': "https://fkuulejsc.mediaconvert.us-east-2.amazonaws.com"
}
first_start = '00:00:00:00'
for i in range(len(data)-1):
    print(data[i]['type'],data[i]['show'])
    if not data[i]['type'] == "machine" and data[i]['show'] == "true":
        print('Processing ' + str(i))
        details_array = []
        print("Record " + str(i))
        start = int(data[i]['start'])
        print(start)
        start_ms = int(start % 1000)
        rest = int((start - start_ms) / 1000)
        start_seconds = int((rest % 60))
        rest = int((rest - start_seconds) / 60)
        start_minute = int(rest % 60)
        start_hours = int((rest - start_minute) / 60)
        duration = int(data[i]['tfs'])
        print(duration)
        end = start + duration
        print(end)
        duration = int(data[i]['tfs'])
        end = start + duration
        end_ms = int(end % 1000)
        rest = int((end - end_ms) / 1000)
        end_seconds = int((rest % 60))
        rest = int((rest - end_seconds) / 60)
        end_minute = int(rest % 60)
        end_hours = int((rest - end_minute) / 60)
        s = (
                str(start_hours).zfill(2) +
                ":" + str(start_minute).zfill(2) +
                ":" + str(start_seconds).zfill(2) +
                ":00"
        )
        print(s)
        e = (
                str(end_hours).zfill(2) +
                ":" + str(end_minute).zfill(2) +
                ":" + str(end_seconds).zfill(2) +
                ":00"
        )
        print(e)
        details_array.append({
            'StartTimecode': s,
            'EndTimecode': e
        })
        media_settings = {
            "OutputGroups": [{
                "Name": "File Group",
                "Outputs": [
                        # output dei frame da analizzare
                        {
                            "ContainerSettings": {
                                "Container": "RAW"
                            },
                            "Extension": ".jpg",
                            "VideoDescription": {
                                "ScalingBehavior": "DEFAULT",
                                "TimecodeInsertion": "DISABLED",
                                "AntiAlias": "ENABLED",
                                "Sharpness": 50,
                                "CodecSettings": {
                                    "Codec": "FRAME_CAPTURE",
                                    "FrameCaptureSettings": {
                                        "FramerateNumerator": 4,
                                        "FramerateDenominator": 1,
                                        "MaxCaptures": 10000000,
                                        "Quality": 80
                                    }
                                },
                                "DropFrameTimecode": "ENABLED",
                                "ColorMetadata": "INSERT"
                            }
                        },
                        # output obbligatorio di almeno 1 video,
                        # scelto volutamente di bassa qualità
                        {
                            "Extension": ".mp4",
                            "NameModifier": "-low",
                            "VideoDescription": {
                                "Width": 852,
                                "ScalingBehavior": "DEFAULT",
                                "Height": 480,
                                "TimecodeInsertion": "DISABLED",
                                "AntiAlias": "ENABLED",
                                "Sharpness": 50,
                                "CodecSettings": {
                                    "Codec": "H_264",
                                    "H264Settings": H264
                                },
                                "AfdSignaling": "NONE",
                                "DropFrameTimecode": "ENABLED",
                                "RespondToAfd": "NONE",
                                "ColorMetadata": "INSERT"
                            },
                            "ContainerSettings": {
                                "Container": "MP4",
                                "Mp4Settings": {
                                    "CslgAtom": "INCLUDE",
                                    "CttsVersion": 0,
                                    "FreeSpaceBox": "EXCLUDE",
                                    "MoovPlacement": "PROGRESSIVE_DOWNLOAD"
                                }
                            }
                        }
                    ],
                    "OutputGroupSettings": {
                        "Type": "FILE_GROUP_SETTINGS",
                        "FileGroupSettings": {
                            "Destination": 's3://ahlconsolebucket/cuts/' + labels[data[i]['label']] + '/'
                        }
                    }
                }
            ],
            "AdAvailOffset": 0,
            "Inputs": [{
                "AudioSelectors": {
                    "Audio Selector 1": {
                        "Offset": 0,
                        "DefaultSelection": "DEFAULT",
                        "ProgramSelection": 1
                    }
                },
                "VideoSelector": {
                    "ColorSpace": "FOLLOW",
                    "Rotate": "DEGREE_0",
                    "AlphaBehavior": "DISCARD"
                },
                "FilterEnable": "AUTO",
                "PsiControl": "USE_PSI",
                "FilterStrength": 0,
                "DeblockFilter": "DISABLED",
                "DenoiseFilter": "DISABLED",
                'FileInput': key,
                "InputClippings": details_array,
                'TimecodeSource': 'SPECIFIEDSTART',
                'TimecodeStart': first_start
            }]
        }
        media_conv = boto3.client("mediaconvert",
            endpoint_url=env_settings['Endpoint'])
        result = media_conv.create_job(
            Role=env_settings["Role"],
            Settings=media_settings,
            AccelerationSettings=env_settings["AccelerationSettings"],
            StatusUpdateInterval="SECONDS_60",
            Priority=0,
            Queue='training'
        )

machine true
machine true
machine true
machine true
user true
Processing 4
Record 4
56780
1234
58014
00:00:56:00
00:00:58:00
machine true
machine true
machine true
machine true


In [ ]:
#n frammentazioni i cui risultati vanno inseriti nel bucket dei dataset
#suddivisi per cartelle come i clipping

In [ ]:
#avvio suite di pulizia dai low file nella cartella del dataset incrementale

In [ ]:
#recupero delle immagini con la stessa struttura data e resize

In [ ]:
#recupero dell'immagine built in di image classification 

In [ ]:
#creazione dei channels di training e validation

In [ ]:
#creazione dell'estimator (con modello il vecchio recuperato) e vio del fit
# esempio
# incr_ic = sagemaker.estimator.Estimator(training_image,
#                                         role,
#                                         train_instance_count=1,
#                                         train_instance_type='ml.p2.xlarge',
#                                         train_volume_size=50,
#                                         train_max_run=360000,
#                                         input_mode='File',
#                                         output_path=s3_output_location,
#                                         sagemaker_session=sess,
#                                         hyperparameters=hyperparams,
#                                         model_uri=ic.model_data) # This parameter will ingest the previous job's model as a new channel
# incr_ic.fit(inputs=data_channels, logs=True)

In [ ]:
#deploy dell'enpoint ed eliminazine del vecchio, passaggio del nuovo a front end

In [ ]:
#chiusura notebook